# W6-CRISPR-Cas3-in-frame-deletion

In Workflow 6, CRISPR-Cas3 is paired with a PCR and Gibson protocol to achieve in-frame deletions22(Figure 7A). Just like in workflow 5, this workflow can be used to knock out specific genes in streptomycetes but with the crucial difference of using Cas3 which has been shown to have higher efficiencies for targeted gene deletions in for example BGCs and random-sized deletion in streptomyces22. Due to homologies, and high GC content, this protocol relies on two steps of Gibson cloning to incorporate the sgRNA and the upstream and downstream repair regions. To get started, users can download the pCRISPR-Cas3 file and the genome file (S. coelicolor A3). StreptoAIM will then generate the necessary primers, plasmid files, and checking primers to verify full gene deletions (S5). 


In [24]:
import sys
import os
from Bio.Restriction import StuI
import pandas as pd
from datetime import datetime

# Ensure the src directory is in the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


from pydna.dseqrecord import Dseqrecord

from streptocad.sequence_loading.sequence_loading import (
    load_and_process_plasmid,
    load_and_process_genome_sequences, 
    check_and_convert_input, 
    annotate_dseqrecord
    
)

from streptocad.utils import polymerase_dict, create_primer_df_from_dict,ProjectDirectory,extract_metadata_to_dataframe
from streptocad.primers.primer_generation import create_idt_order_dataframe, make_primer_records, primers_to_IDT, find_best_check_primers_from_genome
from streptocad.crispr.guideRNAcas3_9_12 import extract_sgRNAs, SgRNAargs
from streptocad.cloning.cas3_plasmid_cloning import generate_cas3_primers, cas3_plasmid_pcrs, assemble_cas3_plasmids
from streptocad.cloning.gibson_cloning import (
    find_up_dw_repair_templates,
    assemble_multiple_plasmids_with_repair_templates_for_deletion,
    update_primer_names
)

## INPUT

In [25]:
# Inputs
# 1 Add genome of choice (genbank, fasta)
path_to_genome = '../../data/genomes/Streptomyces_coelicolor_A3_chromosome.gb'
genome = load_and_process_genome_sequences(path_to_genome)[0]

# 2 Add plasmid 
path_to_plasmid = '../../data/plasmids/pCRISPR_Cas3.gbk'
clean_plasmid = load_and_process_plasmid(path_to_plasmid)
print(clean_plasmid.name)

# 3 Choose genes to knock out (list)
genes_to_KO = ['SCO5087', 'SCO5089', 'SCO5090']
genes_to_KO = ['80000-100000', '4000-7000', '9000-14000','15000-20000']

#### Advanced settings ####
# 4 Filtering metrics for sgRNAs
gc_upper = 0.8
gc_lower = 0.2
off_target_seed = 8
off_target_upper = 10
cas_type='cas3'
number_of_sgRNAs_per_group = 2

# 5 Choose polymerase and target melting temperature
chosen_polymerase = polymerase_dict['Phusion High-Fidelity DNA Polymerase (GC Buffer)']
melting_temperature = 65
primer_concentration = 0.4 
primer_number_increment = 23
flanking_region = 500

# 6 Choose overlapping sequences for our plasmid we can use the following
#As per the article **"CRISPR–Cas9, CRISPRi and CRISPR-BEST-mediated genetic manipulation in streptomycetes"** we need the following oligoes: 
#CGGTTGGTAGGATCGACGGC **-N20-** GTTTTAGAGCTAGAAATAGC
up_homology = Dseqrecord('CGGTTGGTAGGATCGACGGC')
dw_homology = Dseqrecord('GTTTTAGAGCTAGAAATAGC')

pCRISPR-Cas3


In [26]:
print(clean_plasmid.id)


pCRISPR-Cas3


# Computation

In [4]:
target_dict, genes_to_KO, annotation_input = check_and_convert_input(genes_to_KO)

print(annotation_input)
if annotation_input == True:
    genome = annotate_dseqrecord(genome, target_dict)


len(genome.features)

True
this is the target dict [{'region_1_(80000-100000)': [80000, 100000]}, {'region_2_(4000-7000)': [4000, 7000]}, {'region_3_(9000-14000)': [9000, 14000]}, {'region_4_(15000-20000)': [15000, 20000]}]
region_1_(80000-100000) [80000, 100000]
region_2_(4000-7000) [4000, 7000]
region_3_(9000-14000) [9000, 14000]
region_4_(15000-20000) [15000, 20000]


25828

In [5]:

# Initialize SgRNAargs with desired parameters
args = SgRNAargs(genome, 
                genes_to_KO,
                step=['find', 'filter'],
                gc_upper = gc_upper,
                gc_lower = gc_lower,
                off_target_seed = off_target_seed,
                off_target_upper = off_target_upper,
                cas_type=cas_type
                )

sgrna_df = extract_sgRNAs(args)
sgrna_df

sgRNA generated were too smallregion_1_(80000-100000). To incorporate this extent borders. Skipping to next locus tag.


,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
356,NC_003888.3,region_1_(80000-100000),80001,1,1,9290,0.705882,TTC,TCCCATGTGCCGTCGACCGCCCACATCCGCAGCC,TCCCATGT,0
364,NC_003888.3,region_1_(80000-100000),80001,1,1,8869,0.617647,TTC,GTGTTAGCTAAGGCCTCCGCCCGCGAAGGAAAGG,GTGTTAGC,0
63,NC_003888.3,region_1_(80000-100000),80001,1,-1,5345,0.676471,TTC,GGCAATCGAACAGGGCTGGAGCGGACGGTGACGG,GGCAATCG,0
142,NC_003888.3,region_1_(80000-100000),80001,1,-1,11939,0.558824,TTC,CTCAGGACTCTTTCGGGACGTCTTGCACGACGTT,CTCAGGAC,0
146,NC_003888.3,region_1_(80000-100000),80001,1,-1,12029,0.617647,TTC,ACAACAAGGTCGCGGTCGTGACGGGTGGAAGTGC,ACAACAAG,0
...,...,...,...,...,...,...,...,...,...,...,...
426,NC_003888.3,region_1_(80000-100000),80001,1,1,3753,0.588235,TTC,CGGAGCGCTCGCTGAGTTATATACGGATCTGCGC,CGGAGCGC,10
689,NC_003888.3,region_4_(15000-20000),15001,1,1,3850,0.529412,TTC,ACCGAGTTCTTCGCCTTCGTCGACAATGACACCA,ACCGAGTT,10
49,NC_003888.3,region_1_(80000-100000),80001,1,-1,4477,0.647059,TTC,GAGGTGCATTTCACTGCGATTGGCGAGCCGCCCC,GAGGTGCA,10
239,NC_003888.3,region_1_(80000-100000),80001,1,1,19760,0.647059,TTC,GGCTACTGTCCGCAGACCGTGGTGCTGGACGACT,GGCTACTG,10


In [6]:
# Filter the DataFrame to retain only up to 5 sgRNA sequences per locus_tag
filtered_df = sgrna_df.groupby('locus_tag').head(number_of_sgRNAs_per_group)
filtered_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
356,NC_003888.3,region_1_(80000-100000),80001,1,1,9290,0.705882,TTC,TCCCATGTGCCGTCGACCGCCCACATCCGCAGCC,TCCCATGT,0
364,NC_003888.3,region_1_(80000-100000),80001,1,1,8869,0.617647,TTC,GTGTTAGCTAAGGCCTCCGCCCGCGAAGGAAAGG,GTGTTAGC,0
630,NC_003888.3,region_3_(9000-14000),9001,1,1,305,0.647059,TTC,ACCATTGCTCGGCTTCCGCTTCGGCGTCCCACAG,ACCATTGC,1
643,NC_003888.3,region_4_(15000-20000),15001,1,-1,1743,0.676471,TTC,TGCAAGTAGCCGAACACGGCCAGGCCGACGGCAG,TGCAAGTA,1
684,NC_003888.3,region_4_(15000-20000),15001,1,1,4374,0.588235,TTC,CCTCGTTCTGAATGGAGCTCGCCATGGGTCAGTC,CCTCGTTC,1
611,NC_003888.3,region_3_(9000-14000),9001,1,1,2495,0.647059,TTC,GGTTTGGTTGTTCGAGGGCTTGGCCTGCGGTCCG,GGTTTGGT,1
512,NC_003888.3,region_2_(4000-7000),4001,1,1,2196,0.735294,TTC,CTGCAGGAGCCGCTGCTGGCCGGGCAACAGGGTG,CTGCAGGA,1
486,NC_003888.3,region_2_(4000-7000),4001,1,-1,1244,0.617647,TTC,GAAAGCATGGCCTCGTCGACGCCACGGCAGGATT,GAAAGCAT,1


## Output

In [7]:
filtered_df_w_primers = generate_cas3_primers(filtered_df)
filtered_df_w_primers

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,Fwd Primer,Rev Primer
356,NC_003888.3,region_1_(80000-100000),80001,1,1,9290,0.705882,TTC,TCCCATGTGCCGTCGACCGCCCACATCCGCAGCC,TCCCATGT,0,TCCCATGTGCCGTCGACCGCCCACATCCGCAGCCGTCGCcCggCaa...,GGCTGCGGATGTGGGCGGTCGACGGCACATGGGAGTTTCAATCCAC...
364,NC_003888.3,region_1_(80000-100000),80001,1,1,8869,0.617647,TTC,GTGTTAGCTAAGGCCTCCGCCCGCGAAGGAAAGG,GTGTTAGC,0,GTGTTAGCTAAGGCCTCCGCCCGCGAAGGAAAGGGTCGCcCggCaa...,CCTTTCCTTCGCGGGCGGAGGCCTTAGCTAACACGTTTCAATCCAC...
630,NC_003888.3,region_3_(9000-14000),9001,1,1,305,0.647059,TTC,ACCATTGCTCGGCTTCCGCTTCGGCGTCCCACAG,ACCATTGC,1,ACCATTGCTCGGCTTCCGCTTCGGCGTCCCACAGGTCGCcCggCaa...,CTGTGGGACGCCGAAGCGGAAGCCGAGCAATGGTGTTTCAATCCAC...
643,NC_003888.3,region_4_(15000-20000),15001,1,-1,1743,0.676471,TTC,TGCAAGTAGCCGAACACGGCCAGGCCGACGGCAG,TGCAAGTA,1,TGCAAGTAGCCGAACACGGCCAGGCCGACGGCAGGTCGCcCggCaa...,CTGCCGTCGGCCTGGCCGTGTTCGGCTACTTGCAGTTTCAATCCAC...
684,NC_003888.3,region_4_(15000-20000),15001,1,1,4374,0.588235,TTC,CCTCGTTCTGAATGGAGCTCGCCATGGGTCAGTC,CCTCGTTC,1,CCTCGTTCTGAATGGAGCTCGCCATGGGTCAGTCGTCGCcCggCaa...,GACTGACCCATGGCGAGCTCCATTCAGAACGAGGGTTTCAATCCAC...
611,NC_003888.3,region_3_(9000-14000),9001,1,1,2495,0.647059,TTC,GGTTTGGTTGTTCGAGGGCTTGGCCTGCGGTCCG,GGTTTGGT,1,GGTTTGGTTGTTCGAGGGCTTGGCCTGCGGTCCGGTCGCcCggCaa...,CGGACCGCAGGCCAAGCCCTCGAACAACCAAACCGTTTCAATCCAC...
512,NC_003888.3,region_2_(4000-7000),4001,1,1,2196,0.735294,TTC,CTGCAGGAGCCGCTGCTGGCCGGGCAACAGGGTG,CTGCAGGA,1,CTGCAGGAGCCGCTGCTGGCCGGGCAACAGGGTGGTCGCcCggCaa...,CACCCTGTTGCCCGGCCAGCAGCGGCTCCTGCAGGTTTCAATCCAC...
486,NC_003888.3,region_2_(4000-7000),4001,1,-1,1244,0.617647,TTC,GAAAGCATGGCCTCGTCGACGCCACGGCAGGATT,GAAAGCAT,1,GAAAGCATGGCCTCGTCGACGCCACGGCAGGATTGTCGCcCggCaa...,AATCCTGCCGTGGCGTCGACGAGGCCATGCTTTCGTTTCAATCCAC...


In [8]:
amplicons = cas3_plasmid_pcrs(clean_plasmid, filtered_df)

amplicons

[[Amplicon(1489), Amplicon(460)],
 [Amplicon(1489), Amplicon(460)],
 [Amplicon(1489), Amplicon(460)],
 [Amplicon(1489), Amplicon(460)],
 [Amplicon(1489), Amplicon(460)],
 [Amplicon(1489), Amplicon(460)],
 [Amplicon(1489), Amplicon(460)],
 [Amplicon(1489), Amplicon(460)]]

In [9]:
# cut plasmid
assembled_cas3_plasmids = assemble_cas3_plasmids(clean_plasmid, amplicons)
assembled_cas3_plasmids

[Contig(o12766),
 Contig(o12766),
 Contig(o12766),
 Contig(o12766),
 Contig(o12766),
 Contig(o12766),
 Contig(o12766),
 Contig(o12766)]

In [10]:

# Constructing a meaningful name, ID, and description for the assembled plasmid using user input
targeting_info = []
for index, row in filtered_df.iterrows():
    formatted_str = f"pCas3_{row['locus_tag']}({row['sgrna_loc']})"
    targeting_info.append(formatted_str)

for i in range(len(assembled_cas3_plasmids)):
    assembled_cas3_plasmids[i].name = f'{targeting_info[i]}'
    assembled_cas3_plasmids[i].id = assembled_cas3_plasmids[i].name  # Using the same value for ID as for name for simplicity
    assembled_cas3_plasmids[i].description = f'Assembled plasmid targeting {", ".join(genes_to_KO)} for single gene knockout, assembled using StreptoCAD.'


In [11]:
assembled_cas3_plasmids[0].name

'pCas3_region_1_(80000-100000)(9290)'

In [12]:
assembled_cas3_plasmids[0].description

'Assembled plasmid targeting region_1_(80000-100000), region_2_(4000-7000), region_3_(9000-14000), region_4_(15000-20000) for single gene knockout, assembled using StreptoCAD.'

In [13]:
assembled_cas3_plasmids[0].id

'pCas3_region_1_(80000-100000)(9290)'

In [14]:
print_plasmids = False

if print_plasmids: 
    for vector in assembled_cas3_plasmids: 
        vector.write(f"../../data/plasmids/sgRNA_plasmids_pCRISPR–Cas3/{vector.id}.gb")

### IDT primers

In [15]:
primer_records = make_primer_records(filtered_df_w_primers)
primer_records

[Dseqrecord(-52),
 Dseqrecord(-54),
 Dseqrecord(-52),
 Dseqrecord(-54),
 Dseqrecord(-52),
 Dseqrecord(-54),
 Dseqrecord(-52),
 Dseqrecord(-54),
 Dseqrecord(-52),
 Dseqrecord(-54),
 Dseqrecord(-52),
 Dseqrecord(-54),
 Dseqrecord(-52),
 Dseqrecord(-54),
 Dseqrecord(-52),
 Dseqrecord(-54)]

In [16]:
idt_primers_cas3=primers_to_IDT(primer_records)
idt_primers_cas3

,Name,Sequence,Concentration,Purification
0,region_1_(80000-100000)_loc_9290_fwd,TCCCATGTGCCGTCGACCGCCCACATCCGCAGCCGTCGCcCggCaa...,25nm,STD
1,region_1_(80000-100000)_loc_9290_rev,GGCTGCGGATGTGGGCGGTCGACGGCACATGGGAGTTTCAATCCAC...,25nm,STD
2,region_1_(80000-100000)_loc_8869_fwd,GTGTTAGCTAAGGCCTCCGCCCGCGAAGGAAAGGGTCGCcCggCaa...,25nm,STD
3,region_1_(80000-100000)_loc_8869_rev,CCTTTCCTTCGCGGGCGGAGGCCTTAGCTAACACGTTTCAATCCAC...,25nm,STD
4,region_3_(9000-14000)_loc_305_fwd,ACCATTGCTCGGCTTCCGCTTCGGCGTCCCACAGGTCGCcCggCaa...,25nm,STD
5,region_3_(9000-14000)_loc_305_rev,CTGTGGGACGCCGAAGCGGAAGCCGAGCAATGGTGTTTCAATCCAC...,25nm,STD
6,region_4_(15000-20000)_loc_1743_fwd,TGCAAGTAGCCGAACACGGCCAGGCCGACGGCAGGTCGCcCggCaa...,25nm,STD
7,region_4_(15000-20000)_loc_1743_rev,CTGCCGTCGGCCTGGCCGTGTTCGGCTACTTGCAGTTTCAATCCAC...,25nm,STD
8,region_4_(15000-20000)_loc_4374_fwd,CCTCGTTCTGAATGGAGCTCGCCATGGGTCAGTCGTCGCcCggCaa...,25nm,STD
9,region_4_(15000-20000)_loc_4374_rev,GACTGACCCATGGCGAGCTCCATTCAGAACGAGGGTTTCAATCCAC...,25nm,STD


# In frame deletion

In [17]:
in_frame_deletion = False 

In [18]:
if in_frame_deletion:
    # Make repair templates
    print('repair_templates_data')
    repair_templates_data = find_up_dw_repair_templates(genome, 
                                                        genes_to_KO, 
                                                        target_tm=melting_temperature, 
                                                        primer_tm_kwargs={'conc':primer_concentration, 'prodcode':chosen_polymerase} )
    # Digest the plasmids
    print('Digest the plasmids')
    processed_records = [Dseqrecord(record, circular=True).cut(StuI)[0] for record in assembled_cas3_plasmids]

    # Rename them appropriately
    print('Rename them appropriately')

    for i in range(len(processed_records)):
        processed_records[i].name = assembled_cas3_plasmids[i].name

    # Assembly 
    print('assembly_data')

    assembly_data = assemble_multiple_plasmids_with_repair_templates_for_deletion(genes_to_KO, processed_records, 
                                                                                repair_templates_data, 
                                                                                overlap=40)
    # updating the primer names to something systematic.
    update_primer_names(assembly_data)

    # Parse through the primer df
    primer_df = create_primer_df_from_dict(assembly_data)

    #Find_unique primers
    unique_df = primer_df.drop_duplicates(keep='first')
    idt_df = create_idt_order_dataframe(unique_df, concentration="25nm", purification="STD")

    # Contigs
    assembled_contigs = []
    for data in assembly_data: 
        contig_record = data['contig']
        contig_record.id = f"{data['name']}_w_rep"
        contig_record.name = f"{data['name']}_w_rep"

        assembled_contigs.append(contig_record)

    print_plasmids = False

    if print_plasmids:
        # Write to genbank files
        output_directory = "../../data/plasmids/sgRNA_plasmids_pCRISPR–Cas3_with_repair_templates/"
        for contig in assembled_contigs: 
            contig.write(f"{output_directory}/{contig.id}.gb")

In [19]:
if in_frame_deletion:
    integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
    plasmid_metadata_df = extract_metadata_to_dataframe(assembled_contigs,
                                                        clean_plasmid,
                                                        integration_names)
else: 
    integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
    plasmid_metadata_df = extract_metadata_to_dataframe(assembled_cas3_plasmids,
                                                        clean_plasmid,
                                                        integration_names)

plasmid_metadata_df

,plasmid_name,date,original_plasmid,integration,size
0,pCas3_region_1_(80000-100000)(9290),2024-07-23,pCRISPR-Cas3,sgRNA_region_1_(80000-100000)(9290),12766
1,pCas3_region_1_(80000-100000)(8869),2024-07-23,pCRISPR-Cas3,sgRNA_region_1_(80000-100000)(8869),12766
2,pCas3_region_3_(9000-14000)(305),2024-07-23,pCRISPR-Cas3,sgRNA_region_3_(9000-14000)(305),12766
3,pCas3_region_4_(15000-20000)(1743),2024-07-23,pCRISPR-Cas3,sgRNA_region_4_(15000-20000)(1743),12766
4,pCas3_region_4_(15000-20000)(4374),2024-07-23,pCRISPR-Cas3,sgRNA_region_4_(15000-20000)(4374),12766
5,pCas3_region_3_(9000-14000)(2495),2024-07-23,pCRISPR-Cas3,sgRNA_region_3_(9000-14000)(2495),12766
6,pCas3_region_2_(4000-7000)(2196),2024-07-23,pCRISPR-Cas3,sgRNA_region_2_(4000-7000)(2196),12766
7,pCas3_region_2_(4000-7000)(1244),2024-07-23,pCRISPR-Cas3,sgRNA_region_2_(4000-7000)(1244),12766


In [20]:
# Getting checking primers
checking_primers_df = find_best_check_primers_from_genome(genome, 
                                       genes_to_KO, 
                                       flanking_region=flanking_region,
                                       target_tm = melting_temperature, 
                                        primer_concentration = primer_concentration, 
                                        polymerase = chosen_polymerase)
checking_primers_df

,locus tag,f_primer_name,r_primer_name,f_primer_sequences(5-3),r_primer_sequences(5-3),f_tm,r_tm,ta,flanking_region,annealing_temperature,...,homodimer_reverse_tm,homodimer_reverse_deltaG (kcal/mol),heterodimer_tm,heterodimer_deltaG (kcal/mol),hairpin_forward_structure_found,hairpin_forward_tm,hairpin_forward_deltaG (kcal/mol),hairpin_reverse_structure_found,hairpin_reverse_tm,hairpin_reverse_deltaG (kcal/mol)
0,region_3_(9000-14000),region_3_(9000-14000)_fwd_checking_primer,region_3_(9000-14000)_rev_checking_primer,ATCCCGCAGCTGCAGG,GAGGAGCGGCAATGGTCATC,61,63,65,500,65,...,1.445721,-1.026208,9.252334,-1.213024,False,0.0,0.0,False,0.0,0.0
1,region_2_(4000-7000),region_2_(4000-7000)_fwd_checking_primer,region_2_(4000-7000)_rev_checking_primer,GACCGCGCTGGGTG,TCCGGCCGCATCG,59,57,60,500,60,...,22.848051,-5.018625,6.535424,-0.061707,False,0.0,0.0,False,0.0,0.0
2,region_4_(15000-20000),region_4_(15000-20000)_fwd_checking_primer,region_4_(15000-20000)_rev_checking_primer,TTGCTGCAACCTGCCTTG,TAGGTGCCTCCGTCGA,61,58,62,500,62,...,-13.794142,-0.897729,-1.321907,0.948660,False,0.0,0.0,False,0.0,0.0
3,region_1_(80000-100000),region_1_(80000-100000)_fwd_checking_primer,region_1_(80000-100000)_rev_checking_primer,GCCGCTGGCCAAGC,AGGCGTGCGCGTGG,60,62,63,700,63,...,22.176370,-2.921991,11.333376,-0.877576,False,0.0,0.0,False,0.0,0.0


In [21]:
# making the primers into dseqrecords
checking_primers_df_idt = create_idt_order_dataframe(checking_primers_df)
checking_primers_df_idt


,Name,Sequence,Concentration,Purification
0,region_3_(9000-14000)_fwd_checking_primer,ATCCCGCAGCTGCAGG,25nm,STD
1,region_2_(4000-7000)_fwd_checking_primer,GACCGCGCTGGGTG,25nm,STD
2,region_4_(15000-20000)_fwd_checking_primer,TTGCTGCAACCTGCCTTG,25nm,STD
3,region_1_(80000-100000)_fwd_checking_primer,GCCGCTGGCCAAGC,25nm,STD
4,region_3_(9000-14000)_rev_checking_primer,GAGGAGCGGCAATGGTCATC,25nm,STD
5,region_2_(4000-7000)_rev_checking_primer,TCCGGCCGCATCG,25nm,STD
6,region_4_(15000-20000)_rev_checking_primer,TAGGTGCCTCCGTCGA,25nm,STD
7,region_1_(80000-100000)_rev_checking_primer,AGGCGTGCGCGTGG,25nm,STD


In [22]:
if in_frame_deletion: 
    full_idt = pd.concat([idt_primers_cas3,idt_df, checking_primers_df_idt],ignore_index=True)
else: 
    full_idt = pd.concat([idt_primers_cas3,checking_primers_df_idt],ignore_index=True)

full_idt

,Name,Sequence,Concentration,Purification
0,region_1_(80000-100000)_loc_9290_fwd,TCCCATGTGCCGTCGACCGCCCACATCCGCAGCCGTCGCcCggCaa...,25nm,STD
1,region_1_(80000-100000)_loc_9290_rev,GGCTGCGGATGTGGGCGGTCGACGGCACATGGGAGTTTCAATCCAC...,25nm,STD
2,region_1_(80000-100000)_loc_8869_fwd,GTGTTAGCTAAGGCCTCCGCCCGCGAAGGAAAGGGTCGCcCggCaa...,25nm,STD
3,region_1_(80000-100000)_loc_8869_rev,CCTTTCCTTCGCGGGCGGAGGCCTTAGCTAACACGTTTCAATCCAC...,25nm,STD
4,region_3_(9000-14000)_loc_305_fwd,ACCATTGCTCGGCTTCCGCTTCGGCGTCCCACAGGTCGCcCggCaa...,25nm,STD
5,region_3_(9000-14000)_loc_305_rev,CTGTGGGACGCCGAAGCGGAAGCCGAGCAATGGTGTTTCAATCCAC...,25nm,STD
6,region_4_(15000-20000)_loc_1743_fwd,TGCAAGTAGCCGAACACGGCCAGGCCGACGGCAGGTCGCcCggCaa...,25nm,STD
7,region_4_(15000-20000)_loc_1743_rev,CTGCCGTCGGCCTGGCCGTGTTCGGCTACTTGCAGTTTCAATCCAC...,25nm,STD
8,region_4_(15000-20000)_loc_4374_fwd,CCTCGTTCTGAATGGAGCTCGCCATGGGTCAGTCGTCGCcCggCaa...,25nm,STD
9,region_4_(15000-20000)_loc_4374_rev,GACTGACCCATGGCGAGCTCCATTCAGAACGAGGGTTTCAATCCAC...,25nm,STD


## Folder with all the generated I/O

In [23]:

input_files = [
    {"name": "input_genome.gb", "content": genome},
    {"name": "input_plasmid.gb", "content": clean_plasmid}
]


if in_frame_deletion: 
    output_files = [
        {"name": "BEST_w_sgRNAs.gb", "content": assembled_contigs}, # LIST OF Dseqrecords
        {"name": "primer_df.csv", "content": primer_df},
        {"name": "full_idt.csv", "content": full_idt},
        {"name": "sgrna_df.csv", "content": sgrna_df},
        {"name": "filtered_sgrna_df.csv", "content": filtered_df},
        {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},

    ]
else: 
    output_files = [
        {"name": "cBEST_w_sgRNAs.gb", "content": assembled_cas3_plasmids}, # LIST OF Dseqrecords
        {"name": "full_idt.csv", "content": full_idt},
        {"name": "sgrna_df.csv", "content": sgrna_df},
        {"name": "filtered_df.csv", "content": filtered_df},
        {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},
    ]
         
input_values = {
    "genes_to_knockout": genes_to_KO,

    "filtering_metrics": {
        "gc_upper": gc_upper,
        "gc_lower": gc_lower,
        "off_target_seed": off_target_seed,
        "off_target_upper": off_target_upper,
        "cas_type": cas_type,
        "number_of_sgRNAs_per_group": number_of_sgRNAs_per_group,

    },  
    "polymerase_settings": {
        "chosen_polymerase": chosen_polymerase,
        "melting_temperature": melting_temperature,
        "primer_concentration": primer_concentration,
        "primer_number_increment": primer_number_increment,
        "flanking_region": flanking_region
    },
    "overlapping_sequences": {
        "up_homology": str(up_homology),
        "dw_homology": str(dw_homology)
    }
}

# Paths to Markdown files
markdown_file_paths = [
    "../../protocols/conjugation_protcol.md",
    "../../protocols/single_target_crispr_plasmid_protcol.md"

]

timestamp = datetime.utcnow().isoformat()

# Create project directory structure
project_directory = ProjectDirectory(
    project_name=f"CRISPR_cas9_inframe_deletion_workflow_{timestamp}",
    input_files=input_files,
    output_files=output_files,
    input_values=input_values,
    markdown_file_paths=markdown_file_paths
)

# DO You want to save the folder? 
save_zip_folder = False 

if save_zip_folder: 
    # Generate the project directory structure and get the zip content
    zip_content = project_directory.create_directory_structure(create_directories=False)

    # Save the zip file to disk (optional)
    with open("project_structure.zip", "wb") as f:
        f.write(zip_content)